In [1]:
import pandas as pd 
import numpy as np 
import os 
import seaborn as sns 
import matplotlib.pyplot as plt
import math
import scipy.signal as sig
import scipy.stats as stats 

## functions 

In [2]:
# merge zeno video data with bw data 
def merge_bw_zv(bw_df, zv_df):

    print('total bw rows with id in video dataset') 
    print(len(zv_df))

    # merge bw data set rows with zeno videos rows 
        # merge bw data set rows with zeno videos rows 
        # id and date needs to be the same 
        # should only use each brainwalk visit once - once PWS_1 video per person 

    merged_bw_zv = []

    # Loop through each row in av_df
    for index, zv_row in zv_df.iterrows():
   
        current_id = zv_row['id_video']
        current_date = zv_row['visit_date_video']
        zv_row_df = pd.DataFrame([zv_row])

        # Find rows in brainwalk data set with same id and same date as current zv data 
        zv_in_bw_current_id_rows = bw_df[(bw_df['bw_id'] == current_id) & (bw_df['trialdate'] == current_date)]
        #zv_in_bw_current_id_date_rows = zv_in_bw_current_id_rows[zv_in_bw_current_id_rows['visit_date'] == current_date]
   
        if len(zv_in_bw_current_id_rows) == 1: 
            bw_row_to_merge = zv_in_bw_current_id_rows
            # merge bw and zv, store merged row
            merged_row = zv_row_df.merge(bw_row_to_merge, left_on='id_video', right_on='bw_id')
            merged_bw_zv.append(merged_row)
        
        # if more than one row for the id and date, pick one with least na values 
        elif len(zv_in_bw_current_id_rows) > 1:
            bw_row_to_merge = zv_in_bw_current_id_rows.loc[[zv_in_bw_current_id_rows.isna().sum(axis=1).idxmin()]]
            # merge bw and zv, store merged row
            merged_row = zv_row_df.merge(bw_row_to_merge, left_on='id_video', right_on='bw_id')
            merged_bw_zv.append(merged_row)

            print('multiple rows for the id and date combo')
            print(current_id)
            print(current_date)

        else: 
            print('No matching id and daterow from video vs mat')
            print(current_id)
            print(current_date)


    # merge all bw and zv data together 
    merged_bw_zv_df = pd.concat(merged_bw_zv)
    merged_bw_zv_df = merged_bw_zv_df.reset_index(drop=True) # reset index 

    # check same ID for each row 
    print('mismatched zeno video vs brainwalk id')
    print(sum(merged_bw_zv_df['id_video'] != merged_bw_zv_df['bw_id']))

    print('mismatched zeno video vs brainwalk date')
    print(sum(merged_bw_zv_df['visit_date_video'] != merged_bw_zv_df['trialdate']))

    # saved merged df for future reference 
#    merged_bw_zv_df.to_csv(os.path.join(out_path,  'zv_bw_merged_' + task + '.csv'))

    return merged_bw_zv_df

In [3]:
# merge home video data with in person data 
# participants walk at preferred pace at home 

def merge_bw_hv(bw_df, hv_df):

    print('total home videos') 
    print(len(hv_df))

    # filter bw ids dataset to only those included in video data set 
    hv_in_bw_df = bw_df[bw_df['bw_id'].isin(hv_df['id_video'])]
   # hv_in_bw_df.to_csv(os.path.join(out_path, 'hv_id_in_bw_df.csv')) # save excel 

    print('total bw rows with id in video dataset') 
    print(len(hv_in_bw_df))

    # Track used rows from zv_in_bw_df
    used_indices = set()

    # Helper function to find the closest date
    def find_closest_date_unique(row, in_bw_df):
        # Filter rows with the same 'bw_id' and not already used
        filtered_df = in_bw_df[(in_bw_df['bw_id'] == row['id_video'])] # & (~in_bw_df.index.isin(used_indices)) - add after video'] for unique only
        if filtered_df.empty:
            return None
        
        # Find the closest date
        closest_idx = (filtered_df['trialdate'] - row['visit_date_video']).abs().idxmin()
       # used_indices.add(closest_idx)  # Mark the row as used
        return filtered_df.loc[closest_idx]

    # Apply the helper function row-wise
    closest_rows = hv_df.apply(
        lambda row: find_closest_date_unique(row, hv_in_bw_df), axis=1
    )
    
    # Convert the results into a DataFrame
   # closest_rows_df = pd.DataFrame(closest_rows.tolist(), index=hv_df.index)

    # Merge the original `zv_task_df` with `closest_rows_df`
    merged_bw_hv_df = hv_df.merge(closest_rows, left_index=True, right_index=True, suffixes=('', '_closest'))

    # add column for date diff 
    merged_bw_hv_df['bw_hv_abs_date_diff'] = abs(merged_bw_hv_df['trialdate'] - merged_bw_hv_df['visit_date_video'])
    
    # check same ID for each row 
    print('mismatched home video vs brainwalk id')
    print(sum(merged_bw_hv_df['id_video'] != merged_bw_hv_df['bw_id']))
    
    return merged_bw_hv_df

## analysis 

In [4]:
# load brainwalkd data 
bw_df = pd.read_excel(r'C:\Users\mmccu\AppData\Local\Temp\ccsecure\2025_01_24_BrainWalk_AllData_Long_MM.xlsx', 
                     index_col = None, 
                     usecols = ['bw_id', 'record_id', 'trialdate', 'visit_date', 'bingoEHR_EDSS_measure_value', 'msfcEHR_T25FW SPEED AVG',
                                'PWS_velocitycmsecmean', 'FW_velocitycmsecmean'])

In [5]:
# load depth proxy data and convert date 
depth_proxy_path = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\test_velocity_01302025\zeno_all_height_proxy_outputs\depth_proxy_all.csv'
depth_proxy_describe_all = pd.read_csv(depth_proxy_path,
                                      index_col = 0)

In [6]:
# path for scatterplots
output_path = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\test_velocity_01302025\zeno_all_height_proxy_outputs\scatterplots'
if not os.path.exists(output_path): 
    os.makedirs(output_path)

In [7]:
depth_proxy_describe_all.columns

Index(['delta_pix_h_count', 'delta_pix_h_mean', 'delta_pix_h_std',
       'delta_pix_h_min', 'delta_pix_h_25%', 'delta_pix_h_50%',
       'delta_pix_h_75%', 'delta_pix_h_max', 'delta_pix_h_rel_count',
       'delta_pix_h_rel_mean', 'delta_pix_h_rel_std', 'delta_pix_h_rel_min',
       'delta_pix_h_rel_25%', 'delta_pix_h_rel_50%', 'delta_pix_h_rel_75%',
       'delta_pix_h_rel_max', 'delta_pix_h_log_count', 'delta_pix_h_log_mean',
       'delta_pix_h_log_std', 'delta_pix_h_log_min', 'delta_pix_h_log_25%',
       'delta_pix_h_log_50%', 'delta_pix_h_log_75%', 'delta_pix_h_log_max',
       'id_video', 'visit_date_video', 'filename'],
      dtype='object')

In [8]:
# convert dates 
if 'home' in depth_proxy_path: 
   depth_proxy_describe_all['visit_date_video'] = pd.to_datetime(depth_proxy_describe_all['visit_date_video'], errors = 'coerce')
elif 'zeno' in depth_proxy_path:
   depth_proxy_describe_all['visit_date_video'] = pd.to_datetime(depth_proxy_describe_all['visit_date_video'].str.replace('_', '-'), format='%Y-%m-%d')

In [9]:
# add bw data to describe df 
if 'home' in depth_proxy_path: 
    depth_proxy_w_bw = merge_bw_hv(bw_df, depth_proxy_describe_all)
elif 'zeno' in depth_proxy_path:
    depth_proxy_w_bw = merge_bw_zv(bw_df, depth_proxy_describe_all)


total bw rows with id in video dataset
621
No matching id and daterow from video vs mat
BW-0121
2022-07-20 00:00:00
No matching id and daterow from video vs mat
BW-0322
2024-06-10 00:00:00
No matching id and daterow from video vs mat
BW-0322
2024-06-10 00:00:00
mismatched zeno video vs brainwalk id
0
mismatched zeno video vs brainwalk date
0


In [9]:
# homevideo debugging --------------------------------------------------
depth_proxy_w_bw.to_csv(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\test_velocity_01302025\home_all_height_proxy_outputs\merged_home_bw.csv')

depth_proxy_match_id = depth_proxy_w_bw.loc[depth_proxy_w_bw['id_video'] == depth_proxy_w_bw['bw_id']]
depth_proxy_match_id.to_csv(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\test_velocity_01302025\home_all_height_proxy_outputs\match_id_merged_home_bw.csv')


depth_proxy_match_id_date = depth_proxy_match_id.loc[depth_proxy_match_id['bw_hv_abs_date_diff'] <= pd.Timedelta(days = 30)]
depth_proxy_match_id_date = depth_proxy_match_id_date.drop_duplicates(subset=['id_video', 'visit_date_video', 'filename'])
depth_proxy_match_id_date.to_csv(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\test_velocity_01302025\home_all_height_proxy_outputs\depth_proxy_match_id_date.csv')

depth_proxy_w_bw = depth_proxy_match_id_date 
# -----------------------------------------------------------------------

In [10]:
len(depth_proxy_w_bw)

618

In [11]:
def scatterplot_all_vs_one(df, column_name, output_path, folder_name):
    
    for current_col_2 in df.columns:
        clean_df = df.dropna(subset=[column_name, current_col_2])

        # consistent for all plots 
        col_1_data = clean_df[column_name]

        # loop through each other column 
        print(current_col_2)
        col_2_data =  clean_df[current_col_2]

        # scatterplot 
        plt.scatter(col_1_data, col_2_data, alpha = 0.5)
        plt.xlabel(column_name)
        plt.ylabel(current_col_2) 

        print(f"length of clean_df: {len(clean_df)}") 

        if not os.path.exists(os.path.join(output_path, folder_name)): 
            os.makedirs(os.path.join(output_path, folder_name))

        plt.savefig(os.path.join(output_path, folder_name, current_col_2 + '.png')) 
        plt.close()

        # correlation 
        corr_results = stats.spearmanr(col_1_data, col_2_data)
        print(f"r statistic: {corr_results.statistic : .2f}")
        print(f"p value: {corr_results.pvalue :.3f}")
        print('-----------------')
        

In [12]:
# both home and zeno videos
# remove outliers in delta pix h greater than 1 
depth_proxy_w_bw_outrm = depth_proxy_w_bw.loc[depth_proxy_w_bw['delta_pix_h_rel_50%'] <= 1]
depth_proxy_w_bw_outrm['delta_pix_h_rel_50%'].max()

0.93

## Zeno Videos 

In [14]:
# Zeno videos ----------------------------------------------------- 
depth_proxy_w_bw_fw_outrm = depth_proxy_w_bw_outrm.loc[depth_proxy_w_bw_outrm['filename'].str.contains('FW_1')]  
depth_proxy_w_bw_pws_outrm = depth_proxy_w_bw_outrm.loc[depth_proxy_w_bw_outrm['filename'].str.contains('PWS_1')]

In [15]:
print(len(depth_proxy_w_bw_fw_outrm))

312


In [16]:
print(len(depth_proxy_w_bw_pws_outrm))

305


In [17]:
# plot all pws vars vs PWS velocity 
scatterplot_all_vs_one(df = depth_proxy_w_bw_pws_outrm,
                       column_name = 'PWS_velocitycmsecmean', 
                       output_path = output_path, 
                       folder_name = 'pws_velocity_vs_depth_proxies')

delta_pix_h_count
length of clean_df: 302
r statistic: -0.34
p value: 0.000
-----------------
delta_pix_h_mean
length of clean_df: 302
r statistic:  0.41
p value: 0.000
-----------------
delta_pix_h_std
length of clean_df: 302
r statistic:  0.18
p value: 0.001
-----------------
delta_pix_h_min
length of clean_df: 302
r statistic:  0.29
p value: 0.000
-----------------
delta_pix_h_25%
length of clean_df: 302
r statistic:  0.50
p value: 0.000
-----------------
delta_pix_h_50%
length of clean_df: 302
r statistic:  0.52
p value: 0.000
-----------------
delta_pix_h_75%
length of clean_df: 302
r statistic:  0.52
p value: 0.000
-----------------
delta_pix_h_max
length of clean_df: 302
r statistic:  0.15
p value: 0.008
-----------------
delta_pix_h_rel_count
length of clean_df: 302
r statistic: -0.36
p value: 0.000
-----------------
delta_pix_h_rel_mean
length of clean_df: 302
r statistic:  0.55
p value: 0.000
-----------------
delta_pix_h_rel_std
length of clean_df: 301
r statistic:  0.21
p v

DTypePromotionError: The DType <class 'numpy.dtypes.Float64DType'> could not be promoted by <class 'numpy.dtypes.DateTime64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.DateTime64DType'>)

In [19]:
# plot all FW vars vs FW velocity 
scatterplot_all_vs_one(df = depth_proxy_w_bw_fw_outrm,
                       column_name = 'FW_velocitycmsecmean', 
                       output_path = output_path, 
                       folder_name = 'fw_velocity_vs_depth_proxies')

delta_pix_h_count
length of clean_df: 309
r statistic: -0.51
p value: 0.000
-----------------
delta_pix_h_mean
length of clean_df: 309
r statistic:  0.39
p value: 0.000
-----------------
delta_pix_h_std
length of clean_df: 308
r statistic:  0.13
p value: 0.024
-----------------
delta_pix_h_min
length of clean_df: 309
r statistic:  0.29
p value: 0.000
-----------------
delta_pix_h_25%
length of clean_df: 309
r statistic:  0.43
p value: 0.000
-----------------
delta_pix_h_50%
length of clean_df: 309
r statistic:  0.50
p value: 0.000
-----------------
delta_pix_h_75%
length of clean_df: 309
r statistic:  0.42
p value: 0.000
-----------------
delta_pix_h_max
length of clean_df: 309
r statistic:  0.08
p value: 0.157
-----------------
delta_pix_h_rel_count
length of clean_df: 309
r statistic: -0.53
p value: 0.000
-----------------
delta_pix_h_rel_mean
length of clean_df: 309
r statistic:  0.49
p value: 0.000
-----------------
delta_pix_h_rel_std
length of clean_df: 305
r statistic:  0.21
p v

DTypePromotionError: The DType <class 'numpy.dtypes.Float64DType'> could not be promoted by <class 'numpy.dtypes.DateTime64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.DateTime64DType'>)

In [20]:
# plot fw proxy vs T25FW velocity 
scatterplot_all_vs_one(df = depth_proxy_w_bw_fw,
                       column_name = 'msfcEHR_T25FW SPEED AVG', 
                       output_path = output_path, 
                       folder_name = 't25fw_vs_depth_proxies')

delta_pix_h_count
length of clean_df: 235
r statistic:  0.48
p value: 0.000
-----------------
delta_pix_h_mean
length of clean_df: 235
r statistic: -0.30
p value: 0.000
-----------------
delta_pix_h_std
length of clean_df: 234
r statistic: -0.04
p value: 0.566
-----------------
delta_pix_h_min
length of clean_df: 235
r statistic: -0.33
p value: 0.000
-----------------
delta_pix_h_25%
length of clean_df: 235
r statistic: -0.40
p value: 0.000
-----------------
delta_pix_h_50%
length of clean_df: 235
r statistic: -0.42
p value: 0.000
-----------------
delta_pix_h_75%
length of clean_df: 235
r statistic: -0.35
p value: 0.000
-----------------
delta_pix_h_max
length of clean_df: 235
r statistic: -0.03
p value: 0.689
-----------------
delta_pix_h_rel_count
length of clean_df: 235
r statistic:  0.46
p value: 0.000
-----------------
delta_pix_h_rel_mean
length of clean_df: 235
r statistic: -0.48
p value: 0.000
-----------------
delta_pix_h_rel_std
length of clean_df: 230
r statistic: -0.17
p v

DTypePromotionError: The DType <class 'numpy.dtypes.Float64DType'> could not be promoted by <class 'numpy.dtypes.DateTime64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.DateTime64DType'>)

In [21]:
# plot fw proxy vs EDSS velocity 
scatterplot_all_vs_one(df = depth_proxy_w_bw_fw,
                       column_name = 'bingoEHR_EDSS_measure_value', 
                       output_path = output_path, 
                       folder_name = 'edss_vs_depth_proxies')

delta_pix_h_count
length of clean_df: 255
r statistic:  0.39
p value: 0.000
-----------------
delta_pix_h_mean
length of clean_df: 255
r statistic: -0.38
p value: 0.000
-----------------
delta_pix_h_std
length of clean_df: 254
r statistic: -0.07
p value: 0.255
-----------------
delta_pix_h_min
length of clean_df: 255
r statistic: -0.37
p value: 0.000
-----------------
delta_pix_h_25%
length of clean_df: 255
r statistic: -0.48
p value: 0.000
-----------------
delta_pix_h_50%
length of clean_df: 255
r statistic: -0.47
p value: 0.000
-----------------
delta_pix_h_75%
length of clean_df: 255
r statistic: -0.44
p value: 0.000
-----------------
delta_pix_h_max
length of clean_df: 255
r statistic: -0.07
p value: 0.256
-----------------
delta_pix_h_rel_count
length of clean_df: 255
r statistic:  0.41
p value: 0.000
-----------------
delta_pix_h_rel_mean
length of clean_df: 255
r statistic: -0.44
p value: 0.000
-----------------
delta_pix_h_rel_std
length of clean_df: 250
r statistic: -0.10
p v

DTypePromotionError: The DType <class 'numpy.dtypes.Float64DType'> could not be promoted by <class 'numpy.dtypes.DateTime64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.DateTime64DType'>)

## Home Videos 

In [18]:
# plot all home vars vs PWS velocity 
scatterplot_all_vs_one(df = depth_proxy_w_bw_outrm,
                       column_name = 'PWS_velocitycmsecmean', 
                       output_path = output_path, 
                       folder_name = 'pws_velocity_vs_depth_proxies')

delta_pix_h_count
length of clean_df: 59
r statistic: -0.35
p value: 0.006
-----------------
delta_pix_h_mean
length of clean_df: 59
r statistic:  0.01
p value: 0.924
-----------------
delta_pix_h_std
length of clean_df: 58
r statistic: -0.18
p value: 0.170
-----------------
delta_pix_h_min
length of clean_df: 59
r statistic:  0.06
p value: 0.662
-----------------
delta_pix_h_25%
length of clean_df: 59
r statistic:  0.17
p value: 0.211
-----------------
delta_pix_h_50%
length of clean_df: 59
r statistic:  0.09
p value: 0.490
-----------------
delta_pix_h_75%
length of clean_df: 59
r statistic:  0.05
p value: 0.689
-----------------
delta_pix_h_max
length of clean_df: 59
r statistic: -0.24
p value: 0.067
-----------------
delta_pix_h_rel_count
length of clean_df: 59
r statistic: -0.31
p value: 0.016
-----------------
delta_pix_h_rel_mean
length of clean_df: 59
r statistic:  0.32
p value: 0.013
-----------------
delta_pix_h_rel_std
length of clean_df: 58
r statistic:  0.10
p value: 0.437

DTypePromotionError: The DType <class 'numpy.dtypes.Float64DType'> could not be promoted by <class 'numpy.dtypes.DateTime64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.DateTime64DType'>)

In [19]:
# plot all home vars vs FW velocity 
scatterplot_all_vs_one(df = depth_proxy_w_bw_outrm,
                       column_name = 'FW_velocitycmsecmean', 
                       output_path = output_path, 
                       folder_name = 'FW_velocity_vs_depth_proxies')

delta_pix_h_count
length of clean_df: 63
r statistic: -0.11
p value: 0.404
-----------------
delta_pix_h_mean
length of clean_df: 63
r statistic:  0.13
p value: 0.311
-----------------
delta_pix_h_std
length of clean_df: 62
r statistic: -0.01
p value: 0.933
-----------------
delta_pix_h_min
length of clean_df: 63
r statistic:  0.01
p value: 0.952
-----------------
delta_pix_h_25%
length of clean_df: 63
r statistic:  0.22
p value: 0.080
-----------------
delta_pix_h_50%
length of clean_df: 63
r statistic:  0.14
p value: 0.275
-----------------
delta_pix_h_75%
length of clean_df: 63
r statistic:  0.17
p value: 0.184
-----------------
delta_pix_h_max
length of clean_df: 63
r statistic: -0.06
p value: 0.644
-----------------
delta_pix_h_rel_count
length of clean_df: 63
r statistic: -0.14
p value: 0.276
-----------------
delta_pix_h_rel_mean
length of clean_df: 63
r statistic:  0.30
p value: 0.018
-----------------
delta_pix_h_rel_std
length of clean_df: 62
r statistic:  0.15
p value: 0.238

DTypePromotionError: The DType <class 'numpy.dtypes.Float64DType'> could not be promoted by <class 'numpy.dtypes.DateTime64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.DateTime64DType'>)

In [20]:
# plot all home vars vs T25fw velocity 
scatterplot_all_vs_one(df = depth_proxy_w_bw_outrm,
                       column_name = 'msfcEHR_T25FW SPEED AVG', 
                       output_path = output_path, 
                       folder_name = 't25fw_vs_depth_proxies')

delta_pix_h_count
length of clean_df: 61
r statistic:  0.24
p value: 0.068
-----------------
delta_pix_h_mean
length of clean_df: 61
r statistic: -0.07
p value: 0.602
-----------------
delta_pix_h_std
length of clean_df: 60
r statistic:  0.13
p value: 0.313
-----------------
delta_pix_h_min
length of clean_df: 61
r statistic:  0.03
p value: 0.813
-----------------
delta_pix_h_25%
length of clean_df: 61
r statistic: -0.17
p value: 0.194
-----------------
delta_pix_h_50%
length of clean_df: 61
r statistic: -0.05
p value: 0.684
-----------------
delta_pix_h_75%
length of clean_df: 61
r statistic: -0.14
p value: 0.284
-----------------
delta_pix_h_max
length of clean_df: 61
r statistic:  0.16
p value: 0.220
-----------------
delta_pix_h_rel_count
length of clean_df: 61
r statistic:  0.21
p value: 0.103
-----------------
delta_pix_h_rel_mean
length of clean_df: 61
r statistic: -0.26
p value: 0.044
-----------------
delta_pix_h_rel_std
length of clean_df: 60
r statistic: -0.07
p value: 0.604

DTypePromotionError: The DType <class 'numpy.dtypes.Float64DType'> could not be promoted by <class 'numpy.dtypes.DateTime64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.DateTime64DType'>)

In [21]:
# plot all home vars vs T25fw velocity 
scatterplot_all_vs_one(df = depth_proxy_w_bw_outrm,
                       column_name = 'bingoEHR_EDSS_measure_value', 
                       output_path = output_path, 
                       folder_name = 'edss_vs_depth_proxies')

delta_pix_h_count
length of clean_df: 61
r statistic:  0.32
p value: 0.011
-----------------
delta_pix_h_mean
length of clean_df: 61
r statistic: -0.12
p value: 0.371
-----------------
delta_pix_h_std
length of clean_df: 60
r statistic:  0.04
p value: 0.787
-----------------
delta_pix_h_min
length of clean_df: 61
r statistic: -0.15
p value: 0.248
-----------------
delta_pix_h_25%
length of clean_df: 61
r statistic: -0.27
p value: 0.038
-----------------
delta_pix_h_50%
length of clean_df: 61
r statistic: -0.17
p value: 0.199
-----------------
delta_pix_h_75%
length of clean_df: 61
r statistic: -0.10
p value: 0.451
-----------------
delta_pix_h_max
length of clean_df: 61
r statistic:  0.04
p value: 0.744
-----------------
delta_pix_h_rel_count
length of clean_df: 61
r statistic:  0.33
p value: 0.009
-----------------
delta_pix_h_rel_mean
length of clean_df: 61
r statistic: -0.05
p value: 0.680
-----------------
delta_pix_h_rel_std
length of clean_df: 60
r statistic:  0.15
p value: 0.255

DTypePromotionError: The DType <class 'numpy.dtypes.Float64DType'> could not be promoted by <class 'numpy.dtypes.DateTime64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.DateTime64DType'>)